# Agenda

## Create a kubernetes cluster with VMware Enterprise PKS

## Attach the cluster to VMware Tanzu Mission Control
### Authenticate to the new K8s cluster
### Verify connection to the K8s cluster

## Login to VMware Tanzu Mission Control
### Authenticate via tmc CLI
### query cluster groups and clusters
### Attach K8s cluster to Tanzu Mission Control

## Retrospective 
___

## Create PKS Kubernetes Cluster

### Authenticate to your PKS API and review existing clusters

In [ ]:
pks

In [ ]:
APIURL=api.us.azure.bekind.io
APIUSER=scott
APIPASS=changeme
#pks login -a $APIURL -u $APIUSER -p $APIPASS -k

In [ ]:
pks clusters

*Example Output:*

In [ ]:
pks cluster test-demo

In [ ]:
pks plans

### Create the PKS cluster

In [ ]:
pks create-cluster

### Decision time... 

Decide now how you will access the master node(s) of the resulting K8s cluster before you continue.

In my case, I have a domain us.azure.bekind.io into which I will manually create a DNS entry for a load balancer whose backend points to a pool of master node(s).

At the very least, if you specify an IP address in the "external-hostname," you have to have already acquired that public IP address on your cloud provider.  This IP or DNS name should be publically accessible.

In [ ]:
pks create-cluster test-azure --external-hostname test.us.azure.bekind.io --plan small

*Example Output:*

Monitor `pks clusters` until the cluster creation is successful

In [ ]:
pks clusters

Capture the credentials for your new K8s cluster

In [ ]:
pks get-credentials test-azure

Examine your new cluster

In [ ]:
kubectl config use-context test-azure

In [ ]:
kubectl cluster-info

In [ ]:
kubectl get namespaces

Notice above the following namespaces DO NOT exist:
* vmware-system-tmc
* postfacto

# Attach the cluster to Tanzu Mission Control

Login to TMC via https://console.cloud.vmware.com

Download the TMC CLI, make it executable, and move it into your system path

*Cluster names and environment variables*

In [3]:
export TMC_CLUSTER_GROUP=cg-jbrightwell
export TMC_CLUSTER=jbrightwell-c04

In [ ]:
tmc

In [ ]:
# tmc login

*Example Output:*

In [2]:
tmc clustergroup list

  NAME            LABELS                
  cg-01                                 
  cg-02                                 
  cg-03                                 
  cg-04                                 
  cg-05                                 
  cg-06                                 
  cg-07                                 
  cg-08                                 
  cg-09                                 
  cg-10                                 
  cg-jbrightwell                        
  default                               
  keith                                 
  non-prod        environment:non-prod  
  prod            environment:prod      


Notice above the cluster group "cg-jbrightwell" does not exist

In [ ]:
tmc cluster list

Notice above the cluster "jbrightwell-c04" does not exist

In [ ]:
tmc clustergroup templates list

## Create a new cluster group if desired

In [4]:
tmc clustergroup create -n $TMC_CLUSTER_GROUP -t default

ℹ using template "default"
✗ rpc error: code = AlreadyExists desc = Failed to create cluster group 


: 1

In [ ]:
tmc clustergroup list

## Attach the PKS cluster into the new Cluster Group

In [ ]:
tmc cluster attach -g $TMC_CLUSTER_GROUP -n $TMC_CLUSTER

In [ ]:
kubectl apply -f k8s-attach-manifest.yaml

In [ ]:
kubectl get namespaces

Notice the "vmware-system-tmc" namespace has been created.

In [ ]:
kubectl -n vmware-system-tmc get all

In [ ]:
tmc cluster list --group $TMC_CLUSTER_GROUP

*This appears to be a bug.  The above command should scope search to 'cg-jbrightwell'*

In [ ]:
open https://vmww1.tmc.cloud.vmware.com/clusters/$TMC_CLUSTER/global/overview

## Retrospective

In [ ]:
open -a Firefox http://postfacto.us.azure.bekind.io

### Create a Tanzu Mission Control policy managed namespace where the application will run

### ... but first we decide to create a Workspace to manage like-policy Namespaces as one

Here are the templates to use to create the Workspace

In [ ]:
tmc workspace templates list

In [ ]:
export TMC_WORKSPACE=prod-frontend
tmc workspace create -n $TMC_WORKSPACE -d "Production internet accessible apps" -d default

In [ ]:
tmc cluster namespace create -c $TMC_CLUSTER -n postfacto -d "Retrospective application" -k $TMC_WORKSPACE

In [ ]:
kubectl get namespace

### Install postfacto via helm3

In [ ]:
helm3 install --namespace postfacto postfacto \
    --values charts/postfacto/deployment-values.yaml \
    https://github.com/pivotal/postfacto/releases/download/4.1.1/postfacto-0.1.0-beta.tgz
kubectl --namespace postfacto apply -f charts/postfacto/postfacto-ingress-us-azure-bekind-io.yaml

gzip -d charts/postfacto/ingress-tls-us-azure-bekind-io.yaml.gz
kubectl --namespace postfacto apply -f charts/postfacto/ingress-tls-us-azure-bekind-io.yaml
gzip charts/postfacto/ingress-tls-us-azure-bekind-io.yaml

In [ ]:
open https://vmww1.tmc.cloud.vmware.com/clusters/$TMC_CLUSTER/global/overview

In [ ]:
kubectl -n postfacto get all

In [ ]:
kubectl -n postfacto \
  exec $(kubectl -n postfacto get pods -l "app.kubernetes.io/name"=postfacto -o=jsonpath='{.items..metadata.name}') \
  create-admin-user jbrightwell@vmware.com bekind

In [ ]:
open -a Firefox https://postfacto.us.azure.bekind.io/admin

In [ ]:
open -a Firefox https://postfacto.us.azure.bekind.io/retros/tmc

Fin